In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [13]:
tf.reset_default_graph() # 값이 저장되는 특성 때문에 tensorboard 를 동시에 사용할 땐 꼭 기입해 주어야 함!!!

data = pd.read_csv("./mobile-train.csv")
train_data = np.array(data, dtype= np.float32)

nb_classes = len(data["price_range"].unique())
feature = train_data.shape[1]-1

x_data = np.array(data.iloc[:,0:-1],dtype=np.float32)
y_data = np.array(data[["price_range"]], dtype=np.float32)

x = tf.placeholder(tf.float32, [None, 20])
y = tf.placeholder(tf.int32, [None,1])

y_one_hot = tf.one_hot(y,nb_classes)
y_one_hot = tf.reshape(y_one_hot, [-1,nb_classes])

#layer1,2,3 and hypothesis
with tf.name_scope("layer1") as scope:
    w1 = tf.Variable(tf.random_normal([20,10]))
    b1 = tf.Variable(tf.random_normal([10]))
    layer1 = tf.nn.softmax(tf.matmul(x,w1) + b1)
    
with tf.name_scope("layer2") as scope:
    w2 = tf.Variable(tf.random_normal([10,10]))
    b2 = tf.Variable(tf.random_normal([10]))
    layer2 = tf.nn.softmax(tf.matmul(layer1,w2)+b2)
    
with tf.name_scope("layer3") as scope:
    w3 = tf.Variable(tf.random_normal([10,10]))
    b3 = tf.Variable(tf.random_normal([10]))
    layer3 = tf.nn.softmax(tf.matmul(layer2,w3)+b3)
    
with tf.name_scope("layer4") as scope:
    w4 = tf.Variable(tf.random_normal([10,nb_classes]))
    b4 = tf.Variable(tf.random_normal([nb_classes]))
    tf.summary.histogram("weight4", w4)
    tf.summary.histogram("bias4",b4)
    hypothesis = tf.nn.softmax(tf.matmul(layer3, w4)+b4)

# cost and model
cost_i= tf.nn.softmax_cross_entropy_with_logits(logits = tf.matmul(layer3, w4)+b4, labels = y_one_hot)
cost = tf.reduce_mean(cost_i)
tf.summary.scalar("cost", cost)

train1 = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
writer1 = tf.summary.FileWriter("./board/0.1")

train2 = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
writer2 = tf.summary.FileWriter("./board/0.01")

predicted = tf.argmax(hypothesis, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, y_data.flatten()), dtype= tf.float32))
tf.summary.scalar("accuracy", accuracy)

summary = tf.summary.merge_all()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer1.add_graph(sess.graph)
writer2.add_graph(sess.graph)

for i in range(1000):
    s, a,_ = sess.run([summary, accuracy, train1], feed_dict={x:x_data, y:y_data})
    writer1.add_summary(s, global_step=i)
    s, a,_ = sess.run([summary, accuracy, train2], feed_dict={x:x_data, y:y_data})
    writer2.add_summary(s, global_step=i)
    if i%100==0:
        print("정확도 : {:5%}".format(a))



정확도 : 25.000000%
정확도 : 22.900000%
정확도 : 19.100000%
정확도 : 19.100000%
정확도 : 19.700000%
정확도 : 19.700000%
정확도 : 19.700000%
정확도 : 19.700000%
정확도 : 19.700000%
정확도 : 21.799999%


In [10]:
tf.reset_default_graph() 

data = pd.read_csv("./data-titanic.csv", header = None)
train_data = np.array(data, dtype= np.float32)



nb_classes = len(data[0].unique())
feature = train_data.shape[1]-1

x_data = np.array(train_data[:,1:],dtype=np.float32)
y_data = np.array(train_data[:,:1], dtype=np.float32)
x = tf.placeholder(tf.float32, [None, feature])
y = tf.placeholder(tf.float32, [None,1])

#layer1,2,3 and hypothesis
with tf.name_scope("layer1") as scope:
    w1 = tf.Variable(tf.random_normal([feature,10]))
    b1 = tf.Variable(tf.random_normal([10]))
    layer1 = tf.sigmoid(tf.matmul(x,w1) + b1)
    
with tf.name_scope("layer2") as scope:
    w2 = tf.Variable(tf.random_normal([10,10]))
    b2 = tf.Variable(tf.random_normal([10]))
    layer2 = tf.sigmoid(tf.matmul(layer1,w2)+b2)
    
with tf.name_scope("layer3") as scope:
    w3 = tf.Variable(tf.random_normal([10,10]))
    b3 = tf.Variable(tf.random_normal([10]))
    layer3 = tf.sigmoid(tf.matmul(layer2,w3)+b3)
    
with tf.name_scope("layer4") as scope:
    w4 = tf.Variable(tf.random_normal([10,1]))
    b4 = tf.Variable(tf.random_normal([1]))
    hypothesis = tf.sigmoid(tf.matmul(layer3, w4)+b4)

# cost and model
cost = -tf.reduce_mean(y*tf.log(hypothesis)+(1-y)*tf.log(1-hypothesis))
tf.summary.scalar("cost", cost)

train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis>0.5, dtype=np.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, y_data), dtype= tf.float32))

summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./titanic")

sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer.add_graph(sess.graph)

for i in range(1000):
    s, a,_ = sess.run([summary, accuracy, train], feed_dict={x:x_data, y:y_data})
    writer.add_summary(s, global_step=i)

    print("정확도 : {:5%}".format(a))


정확도 : 46.352413%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.61616

정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.616164%
정확도 : 61.61616

In [2]:

import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name="weight_1")
    b1 = tf.Variable(tf.random_normal([2]), name="bias_1")
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)


with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="weight_2")
    b2 = tf.Variable(tf.random_normal([1]), name="bias_2")
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)

# cost/loss function
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, summary, cost_val = sess.run(
            [train, merged_summary, cost], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, cost_val)

    # Accuracy report
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )
    
    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")

"""
Hypothesis:
[[6.1310326e-05]
 [9.9993694e-01]
 [9.9995077e-01]
 [5.9751470e-05]] 
Predicted:
[[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
1.0
"""



0 0.7168676
100 0.69085675
200 0.4987003
300 0.22536185
400 0.122916736
500 0.0791458
600 0.05615393
700 0.042352103
800 0.033301707
900 0.026986767
1000 0.022373727
1100 0.018883027
1200 0.016166903
1300 0.014005062
1400 0.01225182
1500 0.010807454
1600 0.009601359
1700 0.008582579
1800 0.0077132024
1900 0.006964777
2000 0.0063154213
2100 0.0057480196
2200 0.0052491585
2300 0.0048080757
2400 0.0044160266
2500 0.0040659807
2600 0.003752165
2700 0.0034696562
2800 0.003214465
2900 0.002983223
3000 0.0027729843
3100 0.0025813607
3200 0.0024062647
3300 0.0022458504
3400 0.0020985582
3500 0.0019630534
3600 0.0018381808
3700 0.0017228625
3800 0.0016161547
3900 0.0015173084
4000 0.0014256047
4100 0.0013404004
4200 0.0012611565
4300 0.0011873196
4400 0.0011184708
4500 0.0010541915
4600 0.0009941373
4700 0.0009379648
4800 0.00088534516
4900 0.00083605415
5000 0.00078983756
5100 0.0007464565
5200 0.00070570176
5300 0.00066743896
5400 0.000631429
5500 0.00059755263
5600 0.00056566036
5700 0.00053

'\nHypothesis:\n[[6.1310326e-05]\n [9.9993694e-01]\n [9.9995077e-01]\n [5.9751470e-05]] \nPredicted:\n[[0.]\n [1.]\n [1.]\n [0.]] \nAccuracy:\n1.0\n'

In [21]:
y_data.dtype


dtype('int32')